# Visualize adversarial training metrics

### Read and parse logs

In [1]:
from pathlib import Path
import os
import json
import pandas as pd

DATA_PATH = Path('..', 'data_dir')
MAX_PATH = DATA_PATH
LOGS_PATH = Path(MAX_PATH, 'logs', 'report')

In [2]:
LOGS_PATH

PosixPath('/cluster/scratch/mmathys/dl_data/logs/report')

In [3]:
def read_log_file(path):
    # reading the data from the file
    with open(path) as f:
        data = f.read()
        
    content = []
    
    for line in data.split('\n'):
        if len(line):
            content.append(json.loads(line))
    
    return content

In [4]:
logs = {}

In [5]:
for p in os.listdir(LOGS_PATH):
    
    if os.path.isdir(Path(LOGS_PATH, p)):
        logs[p] = read_log_file(Path(LOGS_PATH, p, 'log.txt'))
        

In [6]:
logs.keys()

dict_keys(['007', '1', '01', '03', '001', '3', '003'])

In [14]:
# Prepare data
x_values = []
clean_acc1_values = []
adv_acc1_values = []
test_adv_loss_values = []
test_clean_loss_values = []

for l in logs:
    x_values.append(float('0.'+l))

    clean_acc1_values.append(logs[l][2]['test_acc1'])
    adv_acc1_values.append(logs[l][2]['test_adv_acc1'])
    
    test_adv_loss_values.append(logs[l][2]['test_adv_loss'])
    test_clean_loss_values.append(logs[l][2]['test_loss'])

In [15]:
df = pd.DataFrame({'x': x_values, 
                   'clean_acc1_values': clean_acc1_values, 
                   'adv_acc1_values': adv_acc1_values,
                   'test_clean_loss_values': test_clean_loss_values, 
                   'test_adv_loss_values': test_adv_loss_values})

In [16]:
df

,x,clean_acc1_values,adv_acc1_values,test_clean_loss_values,test_adv_loss_values
0,0.007,94.8,46.8,0.792750,1.655914
1,0.100,32.6,0.0,2.281929,2.382762
2,0.010,95.8,50.8,0.619019,1.520914
3,0.030,83.0,3.0,2.147088,2.381864
4,0.001,98.8,95.0,0.079399,0.171170
5,0.300,22.8,0.0,2.290954,2.371296
6,0.003,98.0,87.4,0.119738,0.419471


In [17]:
df = df.sort_values(by=['x'])
df

,x,clean_acc1_values,adv_acc1_values,test_clean_loss_values,test_adv_loss_values
4,0.001,98.8,95.0,0.079399,0.171170
6,0.003,98.0,87.4,0.119738,0.419471
0,0.007,94.8,46.8,0.792750,1.655914
2,0.010,95.8,50.8,0.619019,1.520914
3,0.030,83.0,3.0,2.147088,2.381864
1,0.100,32.6,0.0,2.281929,2.382762
5,0.300,22.8,0.0,2.290954,2.371296


### Plotting

In [37]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Legend
from bokeh.palettes import Spectral10, Category10
import pandas as pd
from io import BytesIO
from PIL import Image
import base64
from bokeh.models import NumeralTickFormatter
from bokeh.models import BasicTickFormatter

output_notebook()

Loading BokehJS ...

In [40]:
p = figure(width=600, height=300, x_axis_type="log") 

# Clean top-1 accuracy
p.line(df.x, df.clean_acc1_values, line_width=2, color='#1f77b4', legend_label='Clean accuracy')
p.circle(df.x, df.clean_acc1_values, fill_color="white", size=4)

# Adversarial top-1 accuracy
p.line(df.x, df.adv_acc1_values, line_width=2, color='#ff7f0e', legend_label='Adversarial accuracy')
p.circle(df.x, df.adv_acc1_values, color='#ff7f0e', fill_color="white", size=4)

# Axes
p.xaxis.axis_label = "ϵ"
p.xaxis.axis_label_text_font_style = "normal"
p.xaxis.ticker = list(df.x)
p.xaxis.formatter = BasicTickFormatter(use_scientific=False)
#p.xaxis.formatter.use_scientific = False

p.yaxis.axis_label = "Top-1 Accuracy"
p.yaxis.axis_label_text_font_style = "italic"


show(p);